# Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from time import time

%matplotlib inline
%matplotlib notebook
plt.style.use('default')

# Operaciones con los dataframes

### DF 1: Educación de los postulantes

Realizamos un merge de los 3 dataframes de educación de los postulantes que nos fueron provistos, para poder trabajar de manera unificada.

In [2]:
postulantes_educ = pd.read_csv('datos_navent_fiuba/h15_fiuba_1_postulantes_educacion.csv')
postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, on=['idpostulante', 'nombre', 'estado']\
                       , how='outer')

postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, on=['idpostulante', 'nombre', 'estado']\
                       , how='outer')

# Renombramos la columna 'nombre' por algo mas apropiado
postulantes_educ.rename(columns={'nombre':'titulo_univ'}, inplace=True)

postulantes_educ.head()

,idpostulante,titulo_univ,estado
0,ZjlZ,Master,En Curso
1,NdJl,Posgrado,En Curso
2,5kNq,Otro,En Curso
3,8rYD,Master,En Curso
4,1Wvj,Universitario,En Curso


In [3]:
postulantes_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 693641 entries, 0 to 693640
Data columns (total 3 columns):
idpostulante    693641 non-null object
titulo_univ     693641 non-null object
estado          693641 non-null object
dtypes: object(3)
memory usage: 21.2+ MB


Transformamos los strings de los títulos universitarios en variables numéricas.
En principio esto lo realizamos para poder eliminar los IDs duplicados del dataframe. Aquí se contemplan los casos de postulantes que ingresaron con distintos títulos o que el estado de su título varió entre una postulación y la otra. A su vez, esto nos servirá más adelante para aplicar los algoritmos de Machine Learning.

Analizamos los distintos tipos de títulos universitarios que existen.

In [4]:
postulantes_educ.groupby('titulo_univ')['titulo_univ'].count()

titulo_univ
Doctorado               620
Master                10074
Otro                  53363
Posgrado              20624
Secundario           244689
Terciario/Técnico    113421
Universitario        250850
Name: titulo_univ, dtype: int64

Pasamos a variables numéricas cada una de las combinaciones de título y estado de título, a fin de armar un orden de jerarquía de títulos, de manera tal de quedarnos por cada ID del postulante con su título más avanzado.

NOTA: consideramos que 'Otro' es el nivel jerárquico más bajo.

In [5]:
# Otro
postulantes_educ.loc[(postulantes_educ["titulo_univ"] == "Otro"), 'estudios'] = 1
 
# Secundario
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Secundario'), 'estudios'] = 2

# Terciario/ Técnico
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Terciario/Técnico'), 'estudios'] = 3

# Universitario
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Universitario'), 'estudios'] = 4

# Posgrado
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Posgrado'), 'estudios'] = 5

# Master
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Master'), 'estudios'] = 6

# Doctorado
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Doctorado'), 'estudios'] = 7

postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,estudios
218434,JBX2MYp,Universitario,Abandonado,4.0
10606,mzbMKMY,Secundario,Graduado,2.0
205445,kPjOGlb,Secundario,Graduado,2.0
485463,LNwaQxB,Master,Graduado,6.0
455835,96Xeppp,Terciario/Técnico,Abandonado,3.0


Analizamos la cantidad de IDs duplicados

In [6]:
IDs_duplicados = sum(postulantes_educ['idpostulante'].value_counts() > 1)
IDs_duplicados

180088

Ordenamos el dataframe según el orden jerárquico de estudio en forma descendente.

In [7]:
postulantes_educ = postulantes_educ.sort_values('estudios', ascending=False)

Eliminamos todos los IDs duplicados. Considerando que previamente se ordenó el dataframe por orden de estudio, quedará un único ID por postulante con su título más avanzado.

In [8]:
postulantes_educ = postulantes_educ.drop_duplicates('idpostulante')
postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,estudios
332688,lDwjkOz,Secundario,Graduado,2.0
268347,dY9jw5D,Secundario,Graduado,2.0
214543,5OAb2X,Universitario,En Curso,4.0
427476,3NzGYaq,Otro,En Curso,1.0
355185,e3BEvr,Posgrado,En Curso,5.0


Comprobamos que efectivamente no quedaron IDs duplicados.

In [9]:
IDs_duplicados = sum(postulantes_educ['idpostulante'].value_counts() > 1)
IDs_duplicados

0

Creamos una nueva columna que indica si el postulante en cuestión tiene un título en curso o ya se graduó/ abandonó

In [10]:
postulantes_educ['estado'].value_counts()

Graduado      253166
En Curso      148362
Abandonado     46381
Name: estado, dtype: int64

In [11]:
postulantes_educ.loc[(postulantes_educ['estado'] == 'Graduado'), 'esta_estudiando'] = 0
postulantes_educ.loc[(postulantes_educ['estado'] == 'Abandonado'), 'esta_estudiando'] = 0
postulantes_educ.loc[(postulantes_educ['estado'] == 'En Curso'), 'esta_estudiando'] = 1

postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,estudios,esta_estudiando
305653,X9xqmJv,Universitario,Abandonado,4.0,0.0
260389,pzdX1GL,Universitario,En Curso,4.0,1.0
8569,bO4Qbd2,Universitario,En Curso,4.0,1.0
689110,kPjxXdJ,Secundario,Graduado,2.0,0.0
594548,xkPeKAK,Universitario,En Curso,4.0,1.0


Eliminamos las columnas 'titulo_univ' y 'estado', ya que la información relativa a esto ya se encuentra contenida en 'orden_estudio' y en 'esta_estudiando'

In [12]:
postulantes_educ.drop(columns={'titulo_univ', 'estado'}, axis=1, inplace=True)
postulantes_educ.sample(5)

,idpostulante,estudios,esta_estudiando
389645,GNeDRL9,5.0,1.0
214817,PmVGb3a,4.0,1.0
92409,OqjQwjv,4.0,0.0
538202,6r550KL,2.0,0.0
171581,akjoqYZ,2.0,0.0


In [13]:
postulantes_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 447909 entries, 393408 to 530578
Data columns (total 3 columns):
idpostulante       447909 non-null object
estudios           447909 non-null float64
esta_estudiando    447909 non-null float64
dtypes: float64(2), object(1)
memory usage: 13.7+ MB


### DF 2: Nacimiento y sexo de los postulantes

Realizamos un merge de los 3 dataframes de nacimiento y sexo de los postulantes que nos fueron provistos, para poder trabajar de manera unificada.

In [14]:
postulantes_gen_nac = pd.read_csv('datos_navent_fiuba/h15_fiuba_2_postulantes_genero_y_edad.csv')
postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_2_postulantes_genero_y_edad.csv')

postulantes_gen_nac = pd.merge(postulantes_gen_nac, postulantes_gen_nac_2, how='outer')

postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')

postulantes_gen_nac = pd.merge(postulantes_gen_nac, postulantes_gen_nac_2, how='outer')

postulantes_gen_nac.head()

,idpostulante,fechanacimiento,sexo
0,6MM,1985-01-01,MASC
1,Nzz,NaN,NO_DECLARA
2,ZX1,NaN,NO_DECLARA
3,Nq5,NaN,NO_DECLARA
4,ebE,1952-07-07,MASC


In [15]:
postulantes_gen_nac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505382 entries, 0 to 505381
Data columns (total 3 columns):
idpostulante       505382 non-null object
fechanacimiento    478699 non-null object
sexo               505382 non-null object
dtypes: object(3)
memory usage: 15.4+ MB


Analizamos los posibles valores de la columna 'sexo'

In [16]:
postulantes_gen_nac['sexo'].value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
0.0                7
Name: sexo, dtype: int64

Descartamos los registros que figuran con 'sexo' == 0.0, ya que son una pequeñísima parte del set de datos y no brindan información consistente.

In [17]:
postulantes_gen_nac = postulantes_gen_nac[postulantes_gen_nac['sexo'] != '0.0']

Comprobamos que efectivamente fueron eliminados esos registros

In [18]:
postulantes_gen_nac['sexo'].value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
Name: sexo, dtype: int64

Analizamos la cantidad de IDs duplicados

In [19]:
IDs_duplicados = sum(postulantes_gen_nac['idpostulante'].value_counts() > 1)
IDs_duplicados

972

Ordenamos el dataframe según el sexo en forma ascendente.

In [20]:
postulantes_gen_nac = postulantes_gen_nac.sort_values('sexo', ascending=True)

Eliminamos todos los IDs duplicados. Considerando que previamente se ordenó el dataframe por sexo y que 'NO_DECLARA' tiene un orden alfabético menor que 'FEM' y 'MASC', quedará un único ID por postulante con su información más específica de sexo.

Interpretamos que la información concreta del sexo nos será relevante en posteriores análisis, por eso filtramos los IDs duplicados de esta forma.

In [21]:
postulantes_gen_nac = postulantes_gen_nac.drop_duplicates('idpostulante')
postulantes_gen_nac.sample(5)

,idpostulante,fechanacimiento,sexo
321806,owEzXjj,1989-11-20,MASC
192795,rm0WvOb,1985-07-28,MASC
367590,DrpGOaP,1979-01-10,MASC
238828,MVr21qL,1986-10-10,MASC
288682,8MPdw9W,1996-04-11,FEM


Se transforma la columna 'fechanacimiento' en 'edad', mediante una serie de operaciones lógicas.

In [22]:
# Renombramos la columna por algo más apropiado
postulantes_gen_nac = postulantes_gen_nac.rename(columns={'fechanacimiento': 'edad'})

# Obtenemos el año de nacimiento
fecha_nac = postulantes_gen_nac['edad'].str.split('-')
año_nac = fecha_nac.str[0]
postulantes_gen_nac['edad'] = año_nac
postulantes_gen_nac['edad'] = pd.to_numeric(postulantes_gen_nac['edad'], errors='coerce').fillna(0).astype(np.int64)

# Filtramos edades inválidas
edad_min = postulantes_gen_nac['edad'] > 2000
edad_max = postulantes_gen_nac['edad'] < 1950
condicion_final = ((edad_min | edad_max))
postulantes_gen_nac = postulantes_gen_nac[np.logical_not(condicion_final)]

# Obtenemos la edad real según el año actual
postulantes_gen_nac['edad'] = postulantes_gen_nac['edad'].apply(lambda x: 2018-x)

postulantes_gen_nac.sample(5)

,idpostulante,edad,sexo
291632,ow2r8za,22,FEM
414539,lDXopOG,28,FEM
140957,lDb4LAz,28,MASC
263270,OqroQKp,24,MASC
389659,qe6rZ81,32,FEM


Creamos la columna rango_edad para minimizar la cantidad de valores posibles correspondientes a la edad del postulante.

Rangos:
- 18-25
- 26-30
- 31-40
- 41-68

In [23]:
rango_18_25 = (postulantes_gen_nac['edad'] >= 18) & (postulantes_gen_nac['edad'] <= 25)
rango_26_30 = (postulantes_gen_nac['edad'] >= 26) & (postulantes_gen_nac['edad'] <= 30)
rango_31_40 = (postulantes_gen_nac['edad'] >= 31) & (postulantes_gen_nac['edad'] <= 40)
rango_41_68 = (postulantes_gen_nac['edad'] >= 41) & (postulantes_gen_nac['edad'] <= 68)

postulantes_gen_nac.loc[(rango_18_25), 'rango_edad'] = 1
postulantes_gen_nac.loc[(rango_26_30), 'rango_edad'] = 2
postulantes_gen_nac.loc[(rango_31_40), 'rango_edad'] = 3
postulantes_gen_nac.loc[(rango_41_68), 'rango_edad'] = 4

postulantes_gen_nac.sample(5)

,idpostulante,edad,sexo,rango_edad
110218,A3Dq2Jl,31,FEM,3.0
421605,6rJqw4,46,FEM,4.0
79849,dYJ5GJM,26,FEM,2.0
142616,bOWvO1E,33,FEM,3.0
271763,ow2Ak6L,26,MASC,2.0


Eliminamos la columna edad. Consideramos que la información importante se encuentra contenida en el rango edad.

In [24]:
postulantes_gen_nac.drop(columns={'edad'}, axis=1, inplace=True)
postulantes_gen_nac.sample(5)

,idpostulante,sexo,rango_edad
134800,JBe65PJ,FEM,2.0
193079,1QkG0Zz,FEM,3.0
110392,6rQDd4L,FEM,2.0
434776,LNwwKpq,MASC,2.0
354921,Bm5ZO2E,MASC,3.0


In [25]:
postulantes_gen_nac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478001 entries, 165770 to 147639
Data columns (total 3 columns):
idpostulante    478001 non-null object
sexo            478001 non-null object
rango_edad      478001 non-null float64
dtypes: float64(1), object(2)
memory usage: 14.6+ MB


Liberamos memoria para mejorar la performance

In [26]:
postulantes_educ_2=0
postulantes_gen_nac_2=0
fecha_nac=0
año_nac=0
edad_min=0
edad_max=0
condicion_final=0
rango_18_25=0
rango_26_30=0
rango_31_40=0
rango_41_68=0

### Merge de los DF 1 y 2

Se realiza un merge de los dos dataframes con información correspondiente a los postulantes

In [27]:
postulantes = pd.merge(postulantes_educ, postulantes_gen_nac, on='idpostulante', how='inner')

postulantes.head()

,idpostulante,estudios,esta_estudiando,sexo,rango_edad
0,a0XaWD,7.0,0.0,MASC,4.0
1,4reGo5z,7.0,0.0,FEM,2.0
2,X9lpKkb,7.0,0.0,MASC,4.0
3,RzMXJ4E,7.0,0.0,FEM,4.0
4,akOA6b9,7.0,0.0,MASC,4.0


In [28]:
postulantes["sexo"].value_counts()

FEM           230514
MASC          206954
NO_DECLARA       983
Name: sexo, dtype: int64

Pasamos a valor numérico el sexo de los postulantes.

- FEM = 1
- MASC = 2
- NO_DECLARA = 0

In [29]:
postulantes.loc[(postulantes['sexo'] == 'FEM'),'sexo'] = 1

postulantes.loc[(postulantes['sexo'] == 'MASC'),'sexo'] = 2

postulantes.loc[(postulantes['sexo'] == 'NO_DECLARA'), 'sexo'] = 0

postulantes.sample(5)

,idpostulante,estudios,esta_estudiando,sexo,rango_edad
384236,VNrApWN,2.0,0.0,2,1.0
269871,zvxX636,3.0,0.0,1,1.0
358169,dY9OZjq,2.0,0.0,2,4.0
358712,4rdAv4q,2.0,0.0,1,1.0
210506,X9oRPoO,4.0,0.0,1,3.0


### DF 3: Vistas de los avisos

Realizamos un merge de los 3 dataframes de vistas de los postulantes a los anuncios que nos fueron provistos, para poder trabajar de manera unificada.

In [30]:
vistas = pd.read_csv('datos_navent_fiuba/fiuba_3_vistas.csv')
vistas_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_3_vistas.csv')

vistas = pd.merge(vistas, vistas_2, how='outer')

vistas_2 = pd.read_csv('datos_navent_fiuba/h15_fiuba_3_vistas.csv')

vistas = pd.merge(vistas, vistas_2, how='outer')

# Renombramos algunas columnas por algo más apropiado
vistas.rename(columns={'idAviso':'idaviso'}, inplace=True)
vistas.rename(columns={'timestamp':'fechavista'}, inplace=True)

vistas.sample(5)

,idaviso,fechavista,idpostulante
16997532,1112438213,2018-04-03T17:06:18.115-0400,xkvNe8P
10211739,1112176012,2018-04-24T14:00:47.116-0400,2zwMJjV
4846451,1112430528,2018-04-24T15:43:29.996-0400,lD20V2M
12460707,1112415111,2018-04-16T07:35:27.785-0400,6A6vO4
16271164,1112314115,2018-04-03T20:44:20.603-0400,VNrbpPd


In [31]:
vistas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18024382 entries, 0 to 18024381
Data columns (total 3 columns):
idaviso         int64
fechavista      object
idpostulante    object
dtypes: int64(1), object(2)
memory usage: 550.1+ MB


Eliminamos los registros en los cuales un mismo idpostulante visitó un mismo aviso en el mismo día. Esto es para
evitar usuarios que actualizaron muchas veces la página y casos similares.

Para realizar esto, previamente cambiamos el formato de la fecha.

In [32]:
# Modificamos el formato de la fecha vista
lista = vistas['fechavista'].str.split("T")
fecha = lista.str[0]
vistas['fechavista'] = fecha

# Eliminamos los registros arriba mencionados
vistas = vistas.drop_duplicates(['idaviso', 'fechavista', 'idpostulante'])
vistas.head()

,idaviso,fechavista,idpostulante
0,1111780242,2018-02-23,YjVJQ6Z
1,1112263876,2018-02-23,BmVpYoR
2,1112327963,2018-02-23,wVkBzZd
3,1112318643,2018-02-23,OqmP9pv
4,1111903673,2018-02-23,DrpbXDP


Ordenamos el dataframe según la 'fechavista' en forma descendente, para tener las fechas más actuales en primer lugar. De esta forma, al generar la columna cant_vistas, nos quedaremos con la última vista del postulante al aviso.

In [33]:
vistas = vistas.sort_values('fechavista', ascending=False)

Agrupamos por (idaviso, idpostulante) para quedarnos con un único registro por cada par y agregamos la columna cant_vistas, la cual determina cuántas veces un postulante vio un determinado aviso.

El registro que quedará será el más actual, según lo explicado arriba. Como ya tenemos la información que consideramos relevante en la columna 'cant_vistas', nos desentendemos de la columna 'fechavista', que creemos que no aportará información de peso para los algoritmos de Machine Learning.

In [34]:
vistas = vistas.groupby(['idaviso','idpostulante']).size().to_frame('cant_vistas').reset_index()
vistas.sample(5)

,idaviso,idpostulante,cant_vistas
1910458,1807692,BR1ZR,1
5051512,1112423908,NewE8L,1
955024,1806156,VGK6d,1
5174887,1112425807,4rmka1z,1
3808227,1112366438,3NLB5zX,2


In [35]:
vistas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8597915 entries, 0 to 8597914
Data columns (total 3 columns):
idaviso         int64
idpostulante    object
cant_vistas     int64
dtypes: int64(2), object(1)
memory usage: 196.8+ MB


### DF 4: Postulaciones a los avisos

Realizamos un merge de los 2 dataframes de postulaciones a los anuncios que nos fueron provistos, para poder trabajar de manera unificada.

In [36]:
postulaciones = pd.read_csv('datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones_2 = pd.read_csv('datos_navent_fiuba/h15_fiuba_4_postulaciones.csv')

postulaciones = pd.merge(postulaciones, postulaciones_2, how='outer')

postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [37]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6604534 entries, 0 to 6604533
Data columns (total 3 columns):
idaviso             int64
idpostulante        object
fechapostulacion    object
dtypes: int64(1), object(2)
memory usage: 201.6+ MB


### Merge de los DF 3 y 4

Se realiza un merge de los dos dataframes con información correspondiente a las vistas y postulaciones a los anuncios por parte de los postulantes.

In [38]:
modelo = pd.merge(vistas, postulaciones, on=['idpostulante', 'idaviso'], how='outer')

modelo.head()

,idaviso,idpostulante,cant_vistas,fechapostulacion
0,18,BolNL,1.0,NaN
1,48375,RwVdKR,1.0,NaN
2,169730,1KjXB,1.0,NaN
3,169730,2AKzxa,1.0,NaN
4,169730,6LJ64,1.0,NaN


Eliminamos la columna 'fechapostulacion' y la transformamos a una columna binaria llamada 'sepostulo'.

De esta forma, aquellos registros que figuraban con una fecha efectiva en 'fechapostulacion', tendrán un 1 en la nueva columna. En cambio, aquellos registros que figuraban con NaN, tendrán un 0.

In [39]:
modelo.loc[(pd.notna(modelo['fechapostulacion'])), 'fechapostulacion'] = 1   # Not NaN
modelo.loc[(modelo['fechapostulacion'].isnull()), 'fechapostulacion'] = 0    # NaN

modelo.rename(columns={'fechapostulacion':'sepostulo'}, inplace=True)

modelo["sepostulo"].value_counts()

0    6823880
1    6604534
Name: sepostulo, dtype: int64

In [40]:
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13428414 entries, 0 to 13428413
Data columns (total 4 columns):
idaviso         int64
idpostulante    object
cant_vistas     float64
sepostulo       int64
dtypes: float64(1), int64(2), object(1)
memory usage: 512.3+ MB


Eliminamos los idaviso e idpostulante repetidos, para dejar un id único

In [41]:
modelo = modelo.drop_duplicates(['idaviso','idpostulante'])
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13427632 entries, 0 to 13428413
Data columns (total 4 columns):
idaviso         int64
idpostulante    object
cant_vistas     float64
sepostulo       int64
dtypes: float64(1), int64(2), object(1)
memory usage: 512.2+ MB


### DF 6: Detalles de los avisos

Se realiza un merge de los 3 dataframes de detalles de los avisos que nos fueron provistos

In [42]:
avisos_detalles = pd.read_csv('datos_navent_fiuba/h15_fiuba_6_avisos_detalle.csv')
avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_2, how='outer')

avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_2, how='outer')

avisos_detalles.sample(5)

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
13339,1112325280,1,CHEVROLET INCORPORA PERSONAL.,"<p>NUEVO, UNICO E INNOVADOR SISTEMA DE VENTAS....",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ventas,BESTGESTION SA
8107,1112241148,1,Empleada Contable,"<p style=""""><strong><span style="""">Adecco Offi...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Contabilidad,Adecco - Región NORTE & OESTE GBA
25434,1112003378,1,Administrativo Patentamientos Concesionario Au...,<p><strong>Taraborelli Automobile S. A. Conces...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Administración,Taraborelli Automobile S.A.
24874,1112361572,1,Desarrollador Frontend,"<p style=""""><strong><span style="""">IT Resource...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,IT Resources
14562,1110888008,1,Administrativa Compras,<p>Importante Empresa metalúrgica se encuentra...,Capital Federal,NaN,Florencio varela 3215,Full-time,Senior / Semi-Senior,Administración,Eurotecno S.A.


In [43]:
avisos_detalles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25563 entries, 0 to 25562
Data columns (total 11 columns):
idaviso                 25563 non-null int64
idpais                  25563 non-null int64
titulo                  25563 non-null object
descripcion             25563 non-null object
nombre_zona             25563 non-null object
ciudad                  137 non-null object
mapacalle               1854 non-null object
tipo_de_trabajo         25563 non-null object
nivel_laboral           25563 non-null object
nombre_area             25563 non-null object
denominacion_empresa    25556 non-null object
dtypes: int64(2), object(9)
memory usage: 2.3+ MB


Eliminamos los avisos duplicados

In [44]:
avisos_detalles = avisos_detalles.drop_duplicates(['idaviso'])

In [45]:
# Renombramos la columna denominacion_empresa por algo más apropiado
avisos_detalles.rename(columns={'denominacion_empresa':'nombre_empresa'}, inplace=True)

Eliminamos las columnas que consideramos no relevantes. Según la explicación aquí abajo, éstas seran 'idpais', 'ciudad' y 'mapacalle'.

In [46]:
# Analizamos la cantidad de 'idpais' posibles
# Dado que hay un único idpais, eliminaremos esta columna.
avisos_detalles['idpais'].value_counts()

1    24950
Name: idpais, dtype: int64

In [47]:
# Analizamos la cantidad de valores nulos en 'ciudad'
# Dado el gran porcentaje de valores nulos, eliminaremos la columna.
avisos_detalles['ciudad'].isnull().value_counts()

True     24818
False      132
Name: ciudad, dtype: int64

In [48]:
# Analizamos la cantidad de valores nulos en 'mapacalle'
# Dado el gran porcentaje de valores nulos, eliminaremos la columna.
avisos_detalles['mapacalle'].isnull().value_counts()

True     23152
False     1798
Name: mapacalle, dtype: int64

In [49]:
# Eliminamos las columnas arriba mencionadas
avisos_detalles.drop(columns={'idpais', 'ciudad', 'mapacalle'}, axis=1, inplace=True)

Pasamos a valor numérico el nombre zona, según un orden jerárquico asignado por nosotros. Entendemos que Gran Buenos Aires lidera el orden jerárquico, considerando la gran cantidad de registros con los que cuenta. Luego lo siguen Capital Federal y Otros (todas las zonas restantes)

In [50]:
avisos_detalles['nombre_zona'].value_counts()

Gran Buenos Aires              22972
Capital Federal                 1914
Buenos Aires (fuera de GBA)       42
La Plata                           4
GBA Oeste                          3
Mendoza                            3
Ciudad de Mendoza                  3
Rosario                            2
Santa Cruz                         1
Catamarca                          1
Neuquen                            1
San Juan                           1
Santa Fe                           1
Tucuman                            1
Cordoba                            1
Name: nombre_zona, dtype: int64

In [51]:
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Gran Buenos Aires'),'nombre_zona'] = 3
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Capital Federal'),'nombre_zona'] = 2
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Buenos Aires (fuera de GBA)'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'GBA Oeste'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'La Plata'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Ciudad de Mendoza'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Mendoza'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Catamarca'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Rosario'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Santa Cruz'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'San Juan'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Cordoba'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Tucuman'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Neuquen'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Santa Fe'),'nombre_zona'] = 1

Pasamos a valor numérico el tipo de trabajo, según un orden jerárquico asignado por nosotros.

In [52]:
avisos_detalles['tipo_de_trabajo'].value_counts()

Full-time          22523
Part-time           1725
Teletrabajo          248
Por Horas            124
Pasantia             119
Temporario            89
Por Contrato          87
Fines de Semana       28
Primer empleo          6
Voluntario             1
Name: tipo_de_trabajo, dtype: int64

In [53]:
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Full-time'),'tipo_de_trabajo'] = 10
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Part-time'),'tipo_de_trabajo'] = 9
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Teletrabajo'),'tipo_de_trabajo'] = 8
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Horas'),'tipo_de_trabajo'] = 7
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Pasantia'),'tipo_de_trabajo'] = 6
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Temporario'),'tipo_de_trabajo'] = 5
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Contrato'),'tipo_de_trabajo'] = 4
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Fines de Semana'),'tipo_de_trabajo'] = 3
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Primer empleo'),'tipo_de_trabajo'] = 2
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Voluntario'),'tipo_de_trabajo'] = 1

Pasamos a valor numérico el nivel laboral, según un orden jerárquico asignado por nosotros.

In [54]:
avisos_detalles['nivel_laboral'].value_counts()

Senior / Semi-Senior                    16977
Junior                                   4149
Otro                                     1975
Jefe / Supervisor / Responsable          1527
Gerencia / Alta Gerencia / Dirección      322
Name: nivel_laboral, dtype: int64

In [55]:
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Gerencia / Alta Gerencia / Dirección'),'nivel_laboral'] = 5
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Jefe / Supervisor / Responsable'),'nivel_laboral'] = 4
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Senior / Semi-Senior'),'nivel_laboral'] = 3
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Junior'),'nivel_laboral'] = 2
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Otro'),'nivel_laboral'] = 1

In [56]:
avisos_detalles.sample(5)

,idaviso,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,nombre_empresa
4891,1112228625,Carniceros,<p>Importante cadena frigorífica se encuentra ...,3,10,3,Gastronomia,Assistem
20424,1112473579,"Telemarketer Part time, turno tarde para ven...",<p>Importante empresa de Retiro incorporará v...,3,9,3,Telemarketing,RH Talentum
20732,1112477519,CAJERO ADMINISTRATIVO/A PARA IMPORTANTE RETAIL...,"<span style="""">Para empresa del rubro textil e...",3,10,3,Administración,RH Positivo
15392,1112202162,Operarios de Limpieza Técnica,"<p><span lang=""es"">Para importante empresa ubi...",3,10,2,Mantenimiento,RANDSTAD
204,1112408768,Analista de Pricing con experiencia en Consumo...,<p>Importante autoservicio mayorista se encuen...,3,10,3,Desarrollo de Negocios,Consultores de Empresas SRL


In [57]:
avisos_detalles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24950 entries, 0 to 25549
Data columns (total 8 columns):
idaviso            24950 non-null int64
titulo             24950 non-null object
descripcion        24950 non-null object
nombre_zona        24950 non-null int64
tipo_de_trabajo    24950 non-null int64
nivel_laboral      24950 non-null int64
nombre_area        24950 non-null object
nombre_empresa     24943 non-null object
dtypes: int64(4), object(4)
memory usage: 1.7+ MB


Liberamos memoria para mejorar la performance

In [58]:
vistas=0
vista_2=0
lista=0
fecha=0
postulaciones=0
postulaciones_2=0
avisos_detalles_2=0
postulantes_educ=0
postulantes_gen_nac=0

## Merge del modelo con los postulantes y los detalles de los avisos

Cabe destacar que se realiza un inner join en ambos casos, para contar con registros con datos consistentes y sin valores nulos

In [59]:
modelo = pd.merge(modelo, postulantes, on='idpostulante', how='inner')
modelo.sample(5)

,idaviso,idpostulante,cant_vistas,sepostulo,estudios,esta_estudiando,sexo,rango_edad
2316593,1112467682,GNorEPE,1.0,0,2.0,0.0,2,1.0
6347855,1112484808,dYjr2ZA,1.0,0,3.0,1.0,1,1.0
5169295,1112344973,X9Pk3vQ,NaN,1,4.0,1.0,1,1.0
976683,1112457554,jkj5NkO,1.0,0,4.0,0.0,1,1.0
4151921,1112397007,eYj418,NaN,1,3.0,0.0,1,2.0


In [60]:
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10591337 entries, 0 to 10591336
Data columns (total 8 columns):
idaviso            int64
idpostulante       object
cant_vistas        float64
sepostulo          int64
estudios           float64
esta_estudiando    float64
sexo               int64
rango_edad         float64
dtypes: float64(4), int64(3), object(1)
memory usage: 727.2+ MB


In [61]:
modelo = pd.merge(modelo, avisos_detalles, on='idaviso', how='inner')
modelo.sample(5)

,idaviso,idpostulante,cant_vistas,sepostulo,estudios,esta_estudiando,sexo,rango_edad,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,nombre_empresa
4735955,1112344822,BmBLd6v,NaN,1,2.0,0.0,1,2.0,Musimundo SAN MIGUEL necesita Vendedores,"<p> </p><p style=""""><span style="""">Para nustra...",3,10,3,Ventas,Musimundo SA
7902567,1112312285,bOWGXBb,NaN,1,4.0,0.0,2,1.0,ANALISTA DE CONTROL DE GESTION SSR. - BBVA SEG...,"<p style="""">Estamos en la búsqueda de estudian...",3,10,3,Administracion de Seguros,BBVA Francés
966247,1112431304,ZD24BwR,1.0,1,3.0,1.0,1,1.0,RECEPCIONISTA // TELEFONISTA,"<p>Estudio juridico, ubicado en Microcentro, C...",3,9,3,Recepcionista,EPBA
4076938,1112438501,5mdoxvw,1.0,0,4.0,1.0,2,1.0,Ejecutivo de ventas (Forwarder) Zona CABA y Zo...,"<p> </p><p><span style="""">CEVA LOGISTICS, Empr...",3,10,3,Ventas Internacionales/Exportación,Ceva Logistics
4681917,1112432565,ekO6A9W,1.0,0,4.0,1.0,2,1.0,P - Repositores (Zona Tigre),<p>Importante hipermercado de zona norte incor...,3,9,2,Atención al Cliente,Grupo Gestión


In [62]:
modelo.rename(columns={'estudios':'nivel_estudios'},inplace=True)
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9223108 entries, 0 to 9223107
Data columns (total 15 columns):
idaviso            int64
idpostulante       object
cant_vistas        float64
sepostulo          int64
nivel_estudios     float64
esta_estudiando    float64
sexo               int64
rango_edad         float64
titulo             object
descripcion        object
nombre_zona        int64
tipo_de_trabajo    int64
nivel_laboral      int64
nombre_area        object
nombre_empresa     object
dtypes: float64(4), int64(6), object(5)
memory usage: 1.1+ GB


Analizamos la información del modelo final que será utilizado por los algoritmos de Machine Learning.

Asignamos un 1 a aquellos valores NaN de la columna cant_vistas. Esto tiene dos propósitos:

- Mantener la coherencia. Una persona no puede postularse a un aviso sin visualizarlo.
- Evita valores nulos para los algoritmos de Machine Learning.

In [63]:
modelo.loc[(modelo['cant_vistas'].isnull()), 'cant_vistas'] = 1

Comprobamos que efectivamente no quedó ningún registro con cant_vistas con valor nulo.

In [64]:
modelo['cant_vistas'].isnull().sum()

0

In [65]:
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9223108 entries, 0 to 9223107
Data columns (total 15 columns):
idaviso            int64
idpostulante       object
cant_vistas        float64
sepostulo          int64
nivel_estudios     float64
esta_estudiando    float64
sexo               int64
rango_edad         float64
titulo             object
descripcion        object
nombre_zona        int64
tipo_de_trabajo    int64
nivel_laboral      int64
nombre_area        object
nombre_empresa     object
dtypes: float64(4), int64(6), object(5)
memory usage: 1.1+ GB


Reordenamos las columnas para facilitar el manejo del dataframe cuando apliquemos los algoritmos de ML

In [66]:
modelo = modelo[['idaviso','idpostulante','cant_vistas','rango_edad','sexo',\
                 'nivel_estudios','esta_estudiando','titulo','descripcion','tipo_de_trabajo',\
                 'nivel_laboral','nombre_zona','nombre_area','nombre_empresa',\
                 'sepostulo']]

modelo.sample(5)

,idaviso,idpostulante,cant_vistas,rango_edad,sexo,nivel_estudios,esta_estudiando,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_zona,nombre_area,nombre_empresa,sepostulo
1644800,1112425713,3NPJWxQ,1.0,3.0,1,4.0,1.0,Cajera - Part Time PSA (CABA),"<p style=""""><strong><span style="""">¡Estamos bu...",9,2,3,Atención al Cliente,PSA,0
629708,1112223876,vVka8pj,1.0,1.0,1,4.0,1.0,OFICIAL DE BANCA EMPRESAS - MICROCENTRO,"<p><span style="""">Orientamos nuestra búsqueda ...",10,3,3,Comercial,BBVA Francés,0
6023422,1112316757,1QdxaYB,1.0,1.0,1,2.0,0.0,Administrativo de depósito - Vicente López-,"<p style=""""><strong style="""">Grupo Cosmos es u...",10,3,3,Administración,Grupo Cosmos,1
6825525,1112263300,kPX8jAj,1.0,2.0,2,4.0,0.0,Accounting Analysts (Jr/Ssr/Sr),<p><strong>Chevron</strong> is one of the worl...,10,3,3,Contabilidad,Chevron Buenos Aires Shared Services Center,0
4872056,1112389343,ak5YB0N,1.0,3.0,2,4.0,0.0,INGENIERO JUNIOR PARA PLANIFICACION DE OBRAS I...,"<p style=""""><span style="""">Nuestro cliente es ...",10,2,3,Ingeniería Mecánica,Beretta Flavio Danilo,1


In [67]:
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9223108 entries, 0 to 9223107
Data columns (total 15 columns):
idaviso            int64
idpostulante       object
cant_vistas        float64
rango_edad         float64
sexo               int64
nivel_estudios     float64
esta_estudiando    float64
titulo             object
descripcion        object
tipo_de_trabajo    int64
nivel_laboral      int64
nombre_zona        int64
nombre_area        object
nombre_empresa     object
sepostulo          int64
dtypes: float64(4), int64(6), object(5)
memory usage: 1.1+ GB


Analizamos la cantidad de postulaciones y no postulaciones con las que cuenta el dataframe.

In [68]:
modelo["sepostulo"].value_counts()

1    5792310
0    3430798
Name: sepostulo, dtype: int64

Liberamos memoria para mejorar la performance

In [69]:
postulantes=0
avisos_detalles=0

Considerando la gran cantidad de registros que tiene el dataframe, decidimos tomar una porción aleatoria y equitativa de postulaciones y no postulaciones

In [70]:
no = modelo["sepostulo"] == 0
si = modelo["sepostulo"] == 1

modelo_no = modelo[no]
modelo_si = modelo[si]

In [71]:
n_samples = 500000

modelo_no = modelo_no.sample(n_samples)
modelo_si = modelo_si.sample(n_samples)

modelo_final = pd.merge(modelo_no, modelo_si, how='outer')

modelo_final.sample(5)

,idaviso,idpostulante,cant_vistas,rango_edad,sexo,nivel_estudios,esta_estudiando,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_zona,nombre_area,nombre_empresa,sepostulo
856019,1111841524,NzrjlLV,1.0,1.0,1,4.0,1.0,Asesores Comerciales Telefónicos,"<p align=""center""><strong>ASSIST CARD</strong>...",9,3,3,Ventas,ASSIST CARD,1
194372,1110531374,kPjr3bQ,1.0,1.0,1,2.0,0.0,Operador Telefónico,"<p>OPERADORES TELEFÓNICOS</p><p align=""left"">E...",9,2,3,Call Center,Importante Empresa,0
32223,1112404095,OqbokQW,1.0,3.0,1,3.0,0.0,Vendedor/a de salón,"<p><span style="""">Empresa comercial de product...",10,3,3,Atención al Cliente,Ezequiel Pereyra,0
823087,1112246784,JBr2GWb,1.0,1.0,2,2.0,0.0,Ayudante de cocina / Zona Ezeiza,"<p><span style="""">Importante empresa de cateri...",10,3,3,Gastronomia,Manpower,1
711151,1112309380,xkdAYMx,1.0,2.0,2,2.0,0.0,Ejecutivo de Cuentas para Chaco - Corrientes,<p>Importante Multinacional busca <strong>Ejec...,10,3,3,Comercial,Chemsearch SRL,1


Transformamos las variables categóricas de entero, objeto o flotante a categóricas propiamente dichas

In [72]:
modelo_final['rango_edad'] = modelo_final['rango_edad'].astype('category')
modelo_final['sexo'] = modelo_final['sexo'].astype('category') 
modelo_final['nivel_estudios'] = modelo_final['nivel_estudios'].astype('category')
modelo_final['esta_estudiando'] = modelo_final['esta_estudiando'].astype('category')
modelo_final['tipo_de_trabajo'] = modelo_final['tipo_de_trabajo'].astype('category')
modelo_final['nivel_laboral'] = modelo_final['nivel_laboral'].astype('category')
modelo_final['nombre_zona'] = modelo_final['nombre_zona'].astype('category')
modelo_final['sepostulo'] = modelo_final['sepostulo'].astype('category')

In [73]:
modelo_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 15 columns):
idaviso            1000000 non-null int64
idpostulante       1000000 non-null object
cant_vistas        1000000 non-null float64
rango_edad         1000000 non-null category
sexo               1000000 non-null category
nivel_estudios     1000000 non-null category
esta_estudiando    1000000 non-null category
titulo             1000000 non-null object
descripcion        1000000 non-null object
tipo_de_trabajo    1000000 non-null category
nivel_laboral      1000000 non-null category
nombre_zona        1000000 non-null category
nombre_area        1000000 non-null object
nombre_empresa     999939 non-null object
sepostulo          1000000 non-null category
dtypes: category(8), float64(1), int64(1), object(5)
memory usage: 68.7+ MB


Exportamos el modelo final a un CSV file, para desentendernos de todas las operaciones costosas de dataframes a partir de este punto, y poder tener el set de entrenamiento ya listo para los algoritmos de Machine Learning.

In [74]:
modelo_final.to_csv("modelo_final.csv", encoding = "utf-8", index = False)